In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
%matplotlib inline
import time

# Pylians, to calculate power spectrum
import MAS_library as MASL
import Pk_library as PKL

# Brenda lib
from brenda_lib import cySim_lib as cysim
from brenda_lib import pySim_lib as pysim

In [2]:
# Simulation basic settings
h = 0.67
Ngrid = 32
Npart = Ngrid ** 3
lbox = 200  # 502.5/h #502.5/h #128/h

In [3]:
# Define base path
base_path = '/home/msoumad/Git/Aletheia-ML_HF/L200-N32'
snapshot_base_dir = os.path.join(base_path, 'Snapshots')
rockstar_base_dir = os.path.join(base_path, 'Rockstar_Outputs')
output_base_dir = os.path.join(base_path, 'Data_Set')

In [4]:
# Define the columns to be read for the Rockstar output
columns = [
    'Particle_ID', 'Px', 'Py', 'Pz', 'PVx', 'PVy', 'PVz',
    'Halo_ID', 'HaloType', 'Hx', 'Hy', 'Hz', 'HVx', 'HVy', 'HVz'
]

In [5]:
def read_gadget_snapshot(snapshot_file):
    # Read snapshot
    header = pysim.read_header_gadget4(fname=snapshot_file)
    gadget4 = pysim.read_dark_matter_gadget4(fname=snapshot_file)

    # Convert to DataFrame
    snapshot = {
        "Particle_ID": gadget4["ids"],
        "Px": gadget4["pos"][:, 0] / h,
        "Py": gadget4["pos"][:, 1] / h,
        "Pz": gadget4["pos"][:, 2] / h,
        "PVx": gadget4["vel"][:, 0],
        "PVy": gadget4["vel"][:, 1],
        "PVz": gadget4["vel"][:, 2],
    }
    snapshot_df = pd.DataFrame(snapshot)
    snapshot_df.set_index("Particle_ID", inplace=True)
    snapshot_df.sort_index(ascending=True, inplace=True)
    return snapshot_df

In [6]:
def read_rockstar_data(rockstar_file):
    rockstar_df = pd.read_csv(rockstar_file, sep='\s+', low_memory=False, names=columns, skiprows=1)
    rockstar_df['Particle_ID'] = rockstar_df['Particle_ID'].astype(int)
    rockstar_df.set_index('Particle_ID', inplace=True)
    return rockstar_df

In [7]:
def combine_dataframes(snapshot_df, rockstar_df):
    combined_df = snapshot_df.merge(rockstar_df, left_index=True, right_index=True, how='left', suffixes=('', '_halo'))
    combined_df['Host'] = combined_df['Halo_ID'].apply(lambda x: 'yes' if pd.notna(x) else 'no')
    combined_df = combined_df.reset_index()[[
        'Particle_ID', 'Px', 'Py', 'Pz', 'PVx', 'PVy', 'PVz',
        'Host', 'Halo_ID', 'HaloType', 'Hx', 'Hy', 'Hz', 'HVx', 'HVy', 'HVz'
    ]]
    return combined_df

In [8]:
def create_directories(base_dir):
    for subdir in ['Training_Set', 'Testing_Set']:
        for fmt in ['Ascii', 'Bin']:
            os.makedirs(os.path.join(base_dir, subdir, fmt), exist_ok=True)

In [9]:
def process_files(snapshot_file, rockstar_file, output_ascii, output_bin):
    print(f"Processing the file {os.path.basename(snapshot_file)}...")

    snapshot_df = read_gadget_snapshot(snapshot_file)
    rockstar_df = read_rockstar_data(rockstar_file)
    combined_df = combine_dataframes(snapshot_df, rockstar_df)

    # Save as ASCII
    combined_df.to_csv(output_ascii, sep=' ', index=False, na_rep='NaN')

    # Save as binary
    combined_df.to_pickle(output_bin)

    print(f"File {os.path.basename(snapshot_file)} has been processed.")

In [10]:
def execute():
    create_directories(output_base_dir)

    start_time = time.time()

    for i in range(150):
        folder_name = f'output_run{i:04d}'
        snapshot_file = os.path.join(snapshot_base_dir, folder_name, 'snapshot_000')
        rockstar_file = os.path.join(rockstar_base_dir, folder_name, 'Halo_Details.ascii')

        if i < 100:
            output_ascii = os.path.join(output_base_dir, 'Training_Set', 'Ascii', f'{i:04d}.ascii')
            output_bin = os.path.join(output_base_dir, 'Training_Set', 'Bin', f'{i:04d}.bin')
        else:
            output_ascii = os.path.join(output_base_dir, 'Testing_Set', 'Ascii', f'{i:04d}.ascii')
            output_bin = os.path.join(output_base_dir, 'Testing_Set', 'Bin', f'{i:04d}.bin')

        print(f"File processing started for {i:04d}")
        process_files(snapshot_file, rockstar_file, output_ascii, output_bin)
        print(f"File processing completed for {i:04d}")

    end_time = time.time()
    print(f"Data set preparation completed in {end_time - start_time:.2f} seconds.")
    
execute()

File processing started for 0000
Processing the file snapshot_000...
Allocating memory for 32768 particles 0.375Mb
Read data in file #0 for 32768/32768 particles...
File snapshot_000 has been processed.
File processing completed for 0000
File processing started for 0001
Processing the file snapshot_000...
Allocating memory for 32768 particles 0.375Mb
Read data in file #0 for 32768/32768 particles...
File snapshot_000 has been processed.
File processing completed for 0001
File processing started for 0002
Processing the file snapshot_000...
Allocating memory for 32768 particles 0.375Mb
Read data in file #0 for 32768/32768 particles...
File snapshot_000 has been processed.
File processing completed for 0002
File processing started for 0003
Processing the file snapshot_000...
Allocating memory for 32768 particles 0.375Mb
Read data in file #0 for 32768/32768 particles...
File snapshot_000 has been processed.
File processing completed for 0003
File processing started for 0004
Processing the 